# Unit C-Hadoop

- Examples From Video Lecture 

##  Demo HDFS

```
# From windows, connect to the Linux instance
PS:>  docker compose exec hive-server bash

# Basic Commands
$ hdfs dfs –command
OR
$ Hadoop fs –command

# what is the current directory?
$Hadoop fs –ls
 # its /user/root

# make it
$ Hadoop fs –mkdir /user/root

# make a folder
$ Hadoop fs –mkdir /user/root/grades

# upload to Hadoop
$ Hadoop fs –put /datasets/grades/*.tsv  /user/root/grades/

### View on  Web… look at those block sizes.


# copy a file
hadoop fs -cp grades/fall2015.tsv grades/foo.tsv

# combine with cat
Hadoop fs –cat grades/*

# make a file with getmerge – a useful command when a reducer produces more than one fgile
Hadoop fs –getmerge grades/* allgrades.tsv

```

## Demo Connecting to Hive

```
# From windows connect to the hive server
PS:> docker-compose exec hive-server bash

# From linux start the hive client
$ beeline -u jdbc:hive2://localhost:10000/default

# now you are in the Hive client
beeline> Show databases

beeline> Show tables

beeline> Create database

```

## Demo Hive Internal Managed  Tables
```

beeline> !sh hadoop fs -ls /user/root/grades

beeline> !sh hadoop fs -cat /user/root/grades/*

beeline> create table grades_internal ( year INT, semster STRING, course STRING, credits INT, grade STRING) row format delimited fields terminated by ‘\t’;

beeline> show tables;

beeline> select * from grades_internal;

# LOAD DATA
beeline> load data inpath ‘/user/root/grades/*.tsv’ overwrite into table grades_internal;

beeline> select * from grades_internal;

beeline> select sum(credits), year from grades_internal group by year;

beeline> drop table grades_internal;

# data is gone
beeline> !sh hadoop fs -ls /user/hive/warehouse/grades_int


```

## Demo Hive External Tables

```
# re-load some grades into HDFS
beeline> !sh  hadoop fs -put /datasets/grades/fall2015.tsv /user/root/grades

beeline> !sh  hadoop fs -ls /user/root/grades


beeline> create external table grades_ext (
    year int,
    term string,
    course string,
    credits int,
    letter string
) row format delimited
fields terminated by '\t'
location '/user/root/grades';

beeline> select * from grades_ext;

# let’s “insert more data….
beeline> !sh  hadoop fs -put /datasets/grades/fall2016.tsv /user/root/grades


beeline> drop table grades_ext;

# data is still there
beeline> !sh hadoop fs -ls /user/cloudera/grades
```


## Demo of Spark Concepts

In [8]:
import pyspark
from pyspark.sql import SparkSession

In [10]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
        .config("hive.metastore.uris", 
                "thrift://hive-metastore:9083") \
        .enableHiveSupport() \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR") # Keeps the noise down!!!

In [11]:
data = [(1, "Fido", "Dog","SPCA",1),(2, "Felix", "Cat", "SPCA",2),(3, "Rover", "Dog","SPCA",1)]
cols = ["id","name","type","shelter","years_at_shelter"]
pets = spark.createDataFrame(data = data, schema = cols)
pets.show()

+---+-----+----+-------+----------------+
| id| name|type|shelter|years_at_shelter|
+---+-----+----+-------+----------------+
|  1| Fido| Dog|   SPCA|               1|
|  2|Felix| Cat|   SPCA|               2|
|  3|Rover| Dog|   SPCA|               1|
+---+-----+----+-------+----------------+



In [13]:
#Filter then sort, then select
plan = pets.where( pets['years_at_shelter'] ==1).sort("name").select("name","type")
plan.explain()
plan.show()

== Physical Plan ==
*(2) Sort [name#126 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(name#126 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#154]
   +- *(1) Project [name#126, type#127]
      +- *(1) Filter (isnotnull(years_at_shelter#129L) AND (years_at_shelter#129L = 1))
         +- *(1) Scan ExistingRDD[id#125L,name#126,type#127,shelter#128,years_at_shelter#129L]


+-----+----+
| name|type|
+-----+----+
| Fido| Dog|
|Rover| Dog|
+-----+----+



In [16]:
# select, filter then sort
plan = pets.select("name","type").where( pets['years_at_shelter'] ==1).sort("name")
plan.explain()
plan.show()

== Physical Plan ==
*(2) Sort [name#126 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(name#126 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#274]
   +- *(1) Project [name#126, type#127]
      +- *(1) Filter (isnotnull(years_at_shelter#129L) AND (years_at_shelter#129L = 1))
         +- *(1) Scan ExistingRDD[id#125L,name#126,type#127,shelter#128,years_at_shelter#129L]


+-----+----+
| name|type|
+-----+----+
| Fido| Dog|
|Rover| Dog|
+-----+----+



In [17]:
plan1 = pets.select("name","type")
plan2 = plan1.where( pets['years_at_shelter'] ==1)
plan = plan2.sort("name")
plan.explain()
plan.show()

== Physical Plan ==
*(2) Sort [name#126 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(name#126 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#312]
   +- *(1) Project [name#126, type#127]
      +- *(1) Filter (isnotnull(years_at_shelter#129L) AND (years_at_shelter#129L = 1))
         +- *(1) Scan ExistingRDD[id#125L,name#126,type#127,shelter#128,years_at_shelter#129L]


+-----+----+
| name|type|
+-----+----+
| Fido| Dog|
|Rover| Dog|
+-----+----+



## Demo of Hive / Spark Integration

In [18]:
spark.sql("drop table if exists default.grades")
spark.sql("""
create external table default.grades (
  year int,
  semester string,
  course string,
  credits int,
  grade string
) 
row format delimited 
fields terminated by '\t' 
location  'hdfs:///user/root/grades/*.tsv'
""")
spark.sql("select * from default.grades").show()

+----+--------+------+-------+-----+
|year|semester|course|credits|grade|
+----+--------+------+-------+-----+
|2015|    Fall|IST101|      1|    A|
|2015|    Fall|IST195|      3|    A|
|2015|    Fall|IST233|      3|   B+|
|2015|    Fall|SOC101|      3|   A-|
|2015|    Fall|MAT221|      3|    C|
|2016|    Fall|IST346|      3|    A|
|2016|    Fall|CHE111|      4|   A-|
|2016|    Fall|PSY120|      3|   B+|
|2016|    Fall|IST256|      3|    A|
|2016|    Fall|ENG121|      3|   B+|
|2016|  Spring|GEO110|      3|   B+|
|2016|  Spring|MAT222|      3|    A|
|2016|  Spring|SOC121|      3|   C+|
|2016|  Spring|BIO240|      3|   B-|
|2017|  Spring|IST462|      3|    A|
|2017|  Spring|MAT411|      3|    C|
|2017|  Spring|SOC422|      3|   B-|
|2017|  Spring|ENV201|      3|   A-|
+----+--------+------+-------+-----+



In [20]:
spark.sql('''
select sum(credits), year, semester 
from default.grades 
group by year, semester
''').explain()

== Physical Plan ==
*(2) HashAggregate(keys=[year#272, semester#273], functions=[sum(cast(credits#275 as bigint))])
+- Exchange hashpartitioning(year#272, semester#273, 200), ENSURE_REQUIREMENTS, [id=#389]
   +- *(1) HashAggregate(keys=[year#272, semester#273], functions=[partial_sum(cast(credits#275 as bigint))])
      +- Scan hive default.grades [year#272, semester#273, credits#275], HiveTableRelation [`default`.`grades`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Data Cols: [year#272, semester#273, course#274, credits#275, grade#276], Partition Cols: []]




In [ ]:
# Hive Internal Table create stuff.

spark.sql("""drop table if exists default.department""")
spark.sql("""CREATE TABLE default.department(
department_id int ,
department_name string
)    
""")
spark.sql("""
INSERT INTO default.department values (101,"Oncology")    
""")
spark.sql("""
INSERT INTO default.department values (102,"Hematology")    
""")
spark.sql("SELECT * FROM default.department").show()

In [ ]:
spark.sql("create table default.department2 stored as avro as select * from default.department")

In [ ]:
spark.sql("select * from default.department2").show()